In [1]:
from common import *
from structures import *

DAY = 16

In [2]:
show_task(DAY)

<article class="day-desc"><h2>--- Day 16: The Floor Will Be Lava ---</h2><p>With the beam of light completely focused <em>somewhere</em>, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.</p>
<p>Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the <span title="Not anymore, there's a blanket!">beam</span> of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.</p>
<p>Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing <em>empty space</em> (<code>.</code>), <em>mirrors</em> (<code>/</code> and <code>\</code>), and <em>splitters</em> (<code>|</code> and <code>-</code>).</p>
<p>The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into <em>heat</em> to melt the rock in the cavern.</p>
<p>You note the layout of the contraption (your puzzle input). For example:</p>
<pre><code>.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
</code></pre>
<p>The beam enters in the top-left corner from the left and heading to the <em>right</em>. Then, its behavior depends on what it encounters as it moves:</p>
<ul>
<li>If the beam encounters <em>empty space</em> (<code>.</code>), it continues in the same direction.</li>
<li>If the beam encounters a <em>mirror</em> (<code>/</code> or <code>\</code>), the beam is <em>reflected</em> 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a <code>/</code> mirror would continue <em>upward</em> in the mirror's column, while a rightward-moving beam that encounters a <code>\</code> mirror would continue <em>downward</em> from the mirror's column.</li>
<li>If the beam encounters the <em>pointy end of a splitter</em> (<code>|</code> or <code>-</code>), the beam passes through the splitter as if the splitter were <em>empty space</em>. For instance, a rightward-moving beam that encounters a <code>-</code> splitter would continue in the same direction.</li>
<li>If the beam encounters the <em>flat side of a splitter</em> (<code>|</code> or <code>-</code>), the beam is <em>split into two beams</em> going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a <code>|</code> splitter would split into two beams: one that continues <em>upward</em> from the splitter's column and one that continues <em>downward</em> from the splitter's column.</li>
</ul>
<p>Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is <em>energized</em> if that tile has at least one beam pass through it, reflect in it, or split in it.</p>
<p>In the above example, here is how the beam of light bounces around the contraption:</p>
<pre><code>&gt;|&lt;&lt;&lt;\....
|v-.\^....
.v...|-&gt;&gt;&gt;
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
&lt;-&gt;-/vv|..
.|&lt;&lt;&lt;2-|.\
.v//.|.v..
</code></pre>
<p>Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is <em>energized</em> (<code>#</code>) or not (<code>.</code>):</p>
<pre><code>######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
</code></pre>
<p>Ultimately, in this example, <code><em>46</em></code> tiles become <em>energized</em>.</p>
<p>The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, <em>how many tiles end up being energized?</em></p>
</article>

In [57]:
lines = get_test_input_lines(DAY)

In [61]:
lines = get_input_lines(DAY)

In [70]:
map = Map.from_object(lines)
emap = Map(map.h, map.w)
beams = [(0, -1, 0, 1)]
processed_lenses: set[tuple[int, int, int, int]] = set()

def beam_step(map: Map, emap: Map) -> None:
    global beams
    new_beams = []
    for beam in beams:
        y, x, dy, dx = beam
        if x != -1 and y != -1 and x != map.w and y != map.h:
            emap.map[y][x] = '#'
        new_y, new_x = y + dy, x + dx
        if new_y < 0 or new_y >= map.h or new_x < 0 or new_x >= map.w:
            continue
        new_point = map.map[new_y][new_x]
        if new_point == '.':
            new_beams.append((new_y, new_x, dy, dx))
        else:
            key = (new_y, new_x, dy, dx)
            if key in processed_lenses:
                continue
            processed_lenses.add(key)

        if new_point == '|':
            if dx == 0:
                new_beams.append((new_y, new_x, dy, dx))
                continue
            new_beams.append((new_y, new_x, -1, 0))
            new_beams.append((new_y, new_x, 1, 0))
        elif new_point == '-':
            if dy == 0:
                new_beams.append((new_y, new_x, dy, dx))
                continue
            new_beams.append((new_y, new_x, 0, -1))
            new_beams.append((new_y, new_x, 0, 1))
        elif new_point == '/':
            if dx == 0:
                new_beams.append((new_y, new_x, 0, -dy))
            elif dy == 0:
                new_beams.append((new_y, new_x, -dx, 0))
        elif new_point == '\\':
            if dx == 0:
                new_beams.append((new_y, new_x, 0, dy))
            elif dy == 0:
                new_beams.append((new_y, new_x, dx, 0))

    beams = new_beams


# def show_beams(map: Map, beams: list[tuple[int, int, int, int]]) -> None:
#     symbols = {
#         (0, 1): '>',
#         (0, -1): '<',
#         (1, 0): 'v',
#         (-1, 0): '^',
#     }
#     bmap = Map(map.h, map.w)
#     for beam in beams:
#         y, x, dy, dx = beam
#         bmap.map[y][x] = symbols[(dy, dx)]
#     bmap.show()


for i in range(10000):
    beam_step(map, emap)
    if not len(beams):
        print('break at', i)
        break
# emap.show()

result = sum([1 for row in emap.map for c in row if c == '#'])
print('result', result)


break at 697
result 7496


In [56]:
result

7497

In [63]:
send_result(DAY, 1, result)

"That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two] (solved in 68:11)"

In [64]:
show_task(DAY, 2)

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from <em>any edge tile</em> and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)</p>
<p>So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that <em>energizes as many tiles as possible</em>.</p>
<p>In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:</p>
<pre><code>.|&lt;2&lt;\....
|v-v\^....
.v.v.|-&gt;&gt;&gt;
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
&lt;-2-/vv|..
.|&lt;&lt;&lt;2-|.\
.v//.|.v..
</code></pre>
<p>Using this configuration, <code><em>51</em></code> tiles are energized:</p>
<pre><code>.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..
</code></pre>
<p>Find the initial beam configuration that energizes the largest number of tiles; <em>how many tiles are energized in that configuration?</em></p>
</article>

In [71]:
lines = get_test_input_lines(DAY)

In [73]:
lines = get_input_lines(DAY)

In [74]:
map = Map.from_object(lines)

start_points = []
for y in range(map.h):
    start_points.append((y, -1, 0, 1))
    start_points.append((y, map.h, 0, -1))
for x in range(map.w):
    start_points.append((-1, x, 1, 0))
    start_points.append((map.w, x, -1, 0))

max_result = 0
for start_point in start_points:
    emap = Map(map.h, map.w)
    beams = [start_point]
    processed_lenses: set[tuple[int, int, int, int]] = set()
    while True:
        beam_step(map, emap)
        if not len(beams):
            break
    current_result = sum([1 for row in emap.map for c in row if c == '#'])
    max_result = max(max_result, current_result)
print('result', max_result)

result 7932


In [75]:
send_result(DAY, 2, max_result)

"That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 16! (solved in 07:45)"